In [1]:
from pprint import pprint

import gensim.corpora as corpora
import nltk
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import pyLDAvis.gensim_models
from gensim.models import CoherenceModel, LdaMulticore
from tqdm.notebook import tqdm

In [2]:
articles = pd.read_pickle('data/processed_articles.pkl')
comments = pd.read_pickle('data/processed_comments.pkl')

In [3]:
sentiment_translation = {
    'positive': 'положительный',
    'negative': 'отрицательный',
    'neutral': 'нейтральный'
}

emotion_translation = {
    'admiration': 'восхищение',
    'amusement': 'веселье',
    'anger': 'злость',
    'annoyance': 'раздражение',
    'approval': 'одобрение',
    'caring': 'забота',
    'confusion': 'непонимание',
    'curiosity': 'любопытство',
    'desire': 'желание',
    'disappointment': 'разочарование',
    'disapproval': 'неодобрение',
    'disgust': 'отвращение',
    'embarrassment': 'смущение',
    'excitement': 'возбуждение',
    'fear': 'страх',
    'gratitude': 'признательность',
    'grief': 'горе',
    'joy': 'радость',
    'love': 'любовь',
    'nervousness': 'нервозность',
    'optimism': 'оптимизм',
    'pride': 'гордость',
    'realization': 'осознание',
    'relief': 'облегчение',
    'remorse': 'раскаяние',
    'sadness': 'грусть',
    'surprise': 'удивление',
    'neutral': 'нейтральность',
}


comments['sentiment'] = comments['sentiment'].map(sentiment_translation)
comments['emotion'] = comments['emotion'].map(emotion_translation)

In [4]:
N = 30

unigrams = [unigram for tokens in comments['purged_tokens'] for unigram in tokens]
bigrams = [bigram for tokens in comments['purged_tokens'] for bigram in nltk.bigrams(tokens)]
trigrams = [trigram for tokens in comments['purged_tokens'] for trigram in nltk.trigrams(tokens)]

unigram_freq = nltk.FreqDist(unigrams)
bigram_freq = nltk.FreqDist(bigrams)
trigram_freq = nltk.FreqDist(trigrams)

most_common_unigrams = unigram_freq.most_common(N)
most_common_bigrams = bigram_freq.most_common(N)
most_common_trigrams = trigram_freq.most_common(N)


unigrams, unigrams_frequencies = map(list, zip(*most_common_unigrams))

bigrams, bigrams_frequencies = map(list, zip(*most_common_bigrams))
bigrams = list(' - '.join(i) for i in bigrams)

trigrams, trigrams_frequencies = map(list, zip(*most_common_trigrams))
trigrams = list(' - '.join(i) for i in trigrams)

In [5]:
plt.figure(figsize=(12, 8))
sns.barplot(x=unigrams_frequencies, y=unigrams, palette="Oranges_d")
plt.title(f'Топ-{N} униграмм')
plt.xlabel('Частота')
plt.ylabel('Униграммы')
plt.tight_layout()
plt.savefig('results/plots/unigrams.png')
plt.close()

In [6]:
plt.figure(figsize=(12, 8))
sns.barplot(x=bigrams_frequencies, y=bigrams, palette="Blues_d")
plt.title(f'Топ-{N} биграмм')
plt.xlabel('Частота')
plt.ylabel('Биграммы')
plt.tight_layout()
plt.savefig('results/plots/bigrams.png')
plt.close()

In [7]:
plt.figure(figsize=(12, 8))
sns.barplot(x=trigrams_frequencies, y=trigrams, palette="Greens_d")
plt.title(f'Топ-{N} триграмм')
plt.xlabel('Частота')
plt.ylabel('Триграммы')
plt.tight_layout()
plt.savefig('results/plots/trigrams.png')
plt.close()

In [8]:
emotion_sentiment_count = comments.groupby(['sentiment', 'emotion']).size().reset_index(name='counts')

In [9]:
plt.figure(figsize=(12, 8))
sns.barplot(x='sentiment', y='counts', hue='emotion', data=emotion_sentiment_count, palette='bright', estimator=sum)
plt.ylim(0, 1000)
plt.title('Распределение эмоций по тональности')
plt.xlabel('Тональноость')
plt.ylabel('Количество комментариев')
plt.legend(title='Эмоция', loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.savefig('results/plots/emotion_sentiment.png')
plt.close()

In [10]:
emotion_count = comments.groupby('emotion').size().reset_index(name='counts')

In [11]:
plt.figure(figsize=(12, 8))
sns.barplot(y='emotion', x='counts', data=emotion_count.sort_values('counts', ascending=False), palette='viridis')
plt.title('Количество комментариев по эмоциям')
plt.xlabel('Количество комментариев')
plt.ylabel('Эмоция')
plt.tight_layout()
plt.savefig('results/plots/emotion.png')
plt.close()

In [12]:
N = 5

emotions_of_interest = {'злость', 'неодобрение', 'восхищение', 'непонимание'}

filtered_data = comments[comments['emotion'].isin(emotions_of_interest)].groupby(['author', 'emotion']).size().reset_index(name='counts')

anger_data = filtered_data[filtered_data['emotion'] == 'злость'].sort_values('counts', ascending=False).head(N)
disapproval_data = filtered_data[filtered_data['emotion'] == 'неодобрение'].sort_values('counts', ascending=False).head(N)
admiration_data = filtered_data[filtered_data['emotion'] == 'восхищение'].sort_values('counts', ascending=False).head(N)
confusion_data = filtered_data[filtered_data['emotion'] == 'непонимание'].sort_values('counts', ascending=False).head(N)

In [13]:
plt.figure(figsize=(12, 10))

plt.subplot(2, 2, 1)
sns.barplot(x='author', y='counts', data=anger_data, palette='summer')
plt.title('Топ злых комментаторов')
plt.xlabel('Комментаторы')
plt.ylabel('Количество комментариев')


plt.subplot(2, 2, 2)
sns.barplot(x='author', y='counts', data=disapproval_data, palette='autumn')
plt.title('Топ порицающих комментаторов')
plt.xlabel('Комментаторы')
plt.ylabel('')

plt.subplot(2, 2, 3)
sns.barplot(x='author', y='counts', data=admiration_data, palette='winter')
plt.title('Топ восхищающихся комментаторов')
plt.xlabel('Комментаторы')
plt.ylabel('Количество комментариев')

plt.subplot(2, 2, 4)
sns.barplot(x='author', y='counts', data=confusion_data, palette='spring')
plt.title('Топ непонимающих комментаторов')
plt.xlabel('Комментаторы')
plt.ylabel('')


plt.tight_layout()
plt.savefig('results/plots/author_emotion.png')
plt.close()


In [4]:
comments_arr = comments['purged_tokens']
comments_arr.shape[0]

18542

In [5]:
documents = comments_arr.to_list()
id2word = corpora.Dictionary(documents)
corpus = [id2word.doc2bow(doc) for doc in documents]

In [6]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics
    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in tqdm(range(start, limit, step), desc="Calculating Coherence Values"):
        model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    
    return model_list, coherence_values

In [7]:
model_list, coherence_values = compute_coherence_values(
    dictionary=id2word, corpus=corpus, texts=documents, start=2, limit=5, step=1
)

Calculating Coherence Values:   0%|          | 0/3 [00:00<?, ?it/s]

In [33]:
data = pd.DataFrame({
    'Количество тем': list(range(2, 20, 1)),
    'Coherence Value': coherence_values
})

plt.figure(figsize=(12, 8))
sns.lineplot(x='Количество тем', y='Coherence Value', data=data)

plt.title('Зависимость Coherence Value от количества тем')
plt.xlabel('Количество тем')
plt.ylabel('Coherence Value')

plt.tight_layout()
plt.savefig('results/plots/coherence_topic.png')
plt.close()

In [15]:
optimal_model = model_list[0]

In [16]:
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.005*"код" + 0.005*"проблема" + 0.004*"статья" + 0.004*"использовать" + '
  '0.004*"сайт" + 0.004*"проект" + 0.004*"например" + 0.003*"пример" + '
  '0.003*"приложение" + 0.003*"простой"'),
 (1,
  '0.006*"статья" + 0.005*"код" + 0.004*"использовать" + 0.004*"проект" + '
  '0.004*"написать" + 0.003*"приложение" + 0.003*"тип" + 0.003*"сайт" + '
  '0.003*"проблема" + 0.003*"решение"')]


In [12]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(optimal_model, corpus, id2word)
pyLDAvis.save_html(LDAvis_prepared, 'results/topic_modeling_4.html')

# "Программирование и Разработка ПО"
# "Веб-Разработка и Контент"
# "Разработка Пользовательских Интерфейсов"
# "Практические Решения в Разработке"

# "Основы Разработки и Решение Проблем"
# "Разработка и Документация: Проекты и Решения"